# Profitable App Profiles for the App Store and Google Play Markets

My aim in this project is to find mobile app profiles that are profitable for the App Store and Google Play markets.

## Opening and Exploring the Data

As of September 2018, there were approximately 2 million iOS apps available on the App Store, and 2.1 million Android apps on Google Play. I'll start by opening the two data sets and then continue with exploring the data.

In [13]:
# Apple Play Data Set #

from csv import reader

opened_file = open('AppleStore.csv')
read_file = reader(opened_file)
ios_data = list(read_file)
ios_header = ios_data[0]
ios = ios_data[1:]

# Google Play Data Set #

from csv import reader

opened_file = open('googleplaystore.csv')
read_file = reader(opened_file)
android_data = list(read_file)
android_header = android_data[0]
android = android_data[1:]

To make it easier to explore the two data sets, I'll first write a function named `explore_data()` that I can use repeatedly to explore rows in a more readable way.

In [14]:
def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]    
    for row in dataset_slice:
        print(row)
        print('\n') # adds a new (empty) line after each row

    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))

explore_data(android, 0, 4, True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


Number of rows: 10841
Number of columns: 13


## Deleting Wrong Data

The Google Play data set has a dedicated discussion section, and we can see that one of the discussions outlines an error for row 10472.

In [15]:
# Find Row with missing Data

for row in android:
    if len(row) != len(android_header) :
        print(row)
        print('\n') # adds a new (empty) line after each row
        print('Row Number', android.index(row))
        print('\n') # adds a new (empty) line after each row

['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']


Row Number 10472




The row 10472 corresponds to the app Life Made WI-Fi Touchscreen Photo Frame, and the rating is 19. This is clearly off because the maximum rating for a Google Play app is 5 (as mentioned in the discussions section, this problem is caused by a missing value in the `'Category'` column). As a consequence, I'll delete this row.

In [16]:
print (len(android))
del android[10472]
print (len(android))

10841
10840


## Removing Duplicate Entries

In total, there are 1,181 cases where an app occurs more than once.

I don't want to count certain apps more than once when I analyze data, so I need to remove the duplicate entries and keep only one entry per app. 

In [17]:
duplicate_apps = []
unique_apps = []

#Created two lists: one for storing the name of duplicate apps, and one for storing the name of unique apps.

for app in android: #Looped through the android data set
    name = app[0]
    if name in unique_apps: #Saved
        duplicate_apps.append(name)
    else: 
        unique_apps.append(name)
        
print('Number of duplicate apps:', len(duplicate_apps))
print('\n')
print('Examples of duplicate apps:', duplicate_apps[:15])

Number of duplicate apps: 1181


Examples of duplicate apps: ['Quick PDF Scanner + OCR FREE', 'Box', 'Google My Business', 'ZOOM Cloud Meetings', 'join.me - Simple Meetings', 'Box', 'Zenefits', 'Google Ads', 'Google My Business', 'Slack', 'FreshBooks Classic', 'Insightly CRM', 'QuickBooks Accounting: Invoicing & Expenses', 'HipChat - Chat Built for Teams', 'Xero Accounting Software']


In [18]:
#Removing the Duplicates.

print('Expected length:', len(android) - 1181)

#First we I will create a dictionary, where each dictionary key is a unique app name and the corresponding 
#dictionary value is the highest number of reviews of that app.

reviews_max = {}

for app in android: #Looped through the android data set
    name = app[0]
    n_reviews = float(app[3])
    if name in reviews_max and reviews_max[name] < n_reviews:
        reviews_max[name] = n_reviews
    elif name not in reviews_max:
        reviews_max[name] = n_reviews
        
print ('Actual length:',len(reviews_max))


Expected length: 9659
Actual length: 9659


In [19]:
android_clean = [] #which will store our new cleaned data set
already_added = [] #which will just store app names

for app in android: #Looped through the android data set
    name = app[0]
    n_reviews = float(app[3])
    if (n_reviews == reviews_max[name]) and (name not in already_added):
        android_clean.append(app)
        already_added.append(name) #this helps us to keep track of apps that we already added.

print ('Length of android_clean:',len(android_clean)) #check to ensure everything went as expected

Length of android_clean: 9659


## Removing Non-English Apps

The names of some of the apps suggest they are not directed toward an English-speaking audience. I am not interested in keeping these kind of apps, so I'll remove them. 

Also, some English app names use emojis or other symbols (™, — (em dash), – (en dash), etc.) that fall outside of the ASCII range. Because of this, I'll only remove an app if its name has more than three non-ASCII characters.

In [20]:
#Test
name = 'Docs To Go™ Free Office Suite'
occur = 0
for character in name:
        if ord(character) > 127:
            occur = occur + 1
if occur >= 3:
    print('False')
else:
    print('True')

True


In [21]:
english_android=[] 

for app in android_clean: #function to filter out non-English apps from android_clean
    name = app[0]
    occur = 0
    
    for character in name:
        if ord(character) > 127: #If an app name contains a character that is greater than 127, then the app has a non-English name
            occur += 1
    if occur <= 3:
        english_android.append(app)

print('The number of Android Apps in English:',len(english_android))

The number of Android Apps in English: 9614


In [22]:
english_ios=[] 

for app in ios: #function to filter out non-English apps from ios data
    name = app[1]
    occur = 0
    
    for character in name:
        if ord(character) > 127: #If an app name contains a character that is greater than 127, then the app has a non-English name
            occur += 1
    if occur <= 3:
        english_ios.append(app)

print('The number of ios Apps in English:',len(english_ios))

The number of ios Apps in English: 6183


## Isolating the Free Apps

The data sets contain both free and non-free apps, and I'll need to isolate only the free apps for my analysis. Below, I isolate the free apps for both of the data sets.

In [23]:
#Time to isolate the free apps in separate lists

android_final=[]

for app in english_android: #function to filter out free apps from english_android (English list with no duplicates)
    price = app[7]
    
    if price == '0': #If an app price is '0', then the app is free
        android_final.append(app)

print('The number of Free Android Apps:',len(android_final))

ios_final=[]

for app in english_ios: #function to filter out free apps from english_android (English list with no duplicates)
    price = app[4]
    
    if price == '0.0': #If an app price is '0', then the app is free
        ios_final.append(app)

print('The number of Free Ios Apps:',len(ios_final))

The number of Free Android Apps: 8864
The number of Free Ios Apps: 3222


## Most Common Apps by Genre

As I mentioned in the introduction, my aim is to determine the kinds of apps that are likely to attract more users. In this section I focus on finding app profiles that are successdull on both the App Store and Google Play markets.

In [24]:
print (ios_header)
print ('\n')
print (android_header)

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


In [25]:
#Print the Genres for each data set

android_genres = []
ios_genres = []

for app in android_final:
    genre = app[1]
    if genre not in android_genres:
        android_genres.append(genre)
        
print (android_genres)

print ('\n')

for app in ios_final:
    genre = app[11]
    if genre not in ios_genres:
        ios_genres.append(genre)
        
print (ios_genres)

['ART_AND_DESIGN', 'AUTO_AND_VEHICLES', 'BEAUTY', 'BOOKS_AND_REFERENCE', 'BUSINESS', 'COMICS', 'COMMUNICATION', 'DATING', 'EDUCATION', 'ENTERTAINMENT', 'EVENTS', 'FINANCE', 'FOOD_AND_DRINK', 'HEALTH_AND_FITNESS', 'HOUSE_AND_HOME', 'LIBRARIES_AND_DEMO', 'LIFESTYLE', 'GAME', 'FAMILY', 'MEDICAL', 'SOCIAL', 'SHOPPING', 'PHOTOGRAPHY', 'SPORTS', 'TRAVEL_AND_LOCAL', 'TOOLS', 'PERSONALIZATION', 'PRODUCTIVITY', 'PARENTING', 'WEATHER', 'VIDEO_PLAYERS', 'NEWS_AND_MAGAZINES', 'MAPS_AND_NAVIGATION']


['Social Networking', 'Photo & Video', 'Games', 'Music', 'Reference', 'Health & Fitness', 'Weather', 'Utilities', 'Travel', 'Shopping', 'News', 'Navigation', 'Lifestyle', 'Entertainment', 'Food & Drink', 'Sports', 'Book', 'Finance', 'Education', 'Productivity', 'Business', 'Catalogs', 'Medical']


In [26]:
# I need to build a frequency table for the prime_genre column of the App Store data set, 
# and for the Genres and Category columns of the Google Play data set.

#I'll build two functions we can use to analyze the frequency tables:

# One function to generate frequency tables that show percentages
# Another function we can use to display the percentages in a descending order

def freq_table(dataset,index):
    table = {}
    total = 0
    
    for row in dataset:
        value = row[index] #we want the name of the category/genre
        total += 1 #represents the total number(the denominator for percentage)
        
        if value in table:
            table[value] += 1
        else: 
            table[value] = 1
    
    table_percentages = {}
    
    for key in table:
        percentage = (table[key]/total)*100 #using the percentages as values
        table_percentages[key] = percentage 
    
    return table_percentages #will use the new dictionary with percentages

def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)

    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])

In [27]:
#Now I will display the frequency table of the columns prime_genre, Genres, and Category.

print('Ios frequency table of the column prime_genre:')
print('\n')
display_table(ios_final,11)

print('Andriod frequency table of the column Category:')
print('\n')
display_table(android_final,1)
print('\n')

Ios frequency table of the column prime_genre:


Games : 58.16263190564867
Entertainment : 7.883302296710118
Photo & Video : 4.9658597144630665
Education : 3.662321539416512
Social Networking : 3.2898820608317814
Shopping : 2.60707635009311
Utilities : 2.5139664804469275
Sports : 2.1415270018621975
Music : 2.0484171322160147
Health & Fitness : 2.0173805090006205
Productivity : 1.7380509000620732
Lifestyle : 1.5828677839851024
News : 1.3345747982619491
Travel : 1.2414649286157666
Finance : 1.1173184357541899
Weather : 0.8690254500310366
Food & Drink : 0.8069522036002483
Reference : 0.5586592178770949
Business : 0.5276225946617008
Book : 0.4345127250155183
Navigation : 0.186219739292365
Medical : 0.186219739292365
Catalogs : 0.12414649286157665
Andriod frequency table of the column Category:


FAMILY : 18.907942238267147
GAME : 9.724729241877256
TOOLS : 8.461191335740072
BUSINESS : 4.591606498194946
LIFESTYLE : 3.9034296028880866
PRODUCTIVITY : 3.892148014440433
FINANCE : 3.7003610108303

The general impression is that App Store (at least the part containing free English apps) is dominated by apps that are designed for fun (games, entertainment, photo and video, social networking, sports, music, etc.), while apps with practical purposes (education, shopping, utilities, productivity, lifestyle, etc.) are more rare. 

The landscape seems significantly different on Google Play: there are not that many apps designed for fun, and it seems that a good number of apps are designed for practical purposes (family, tools, business, lifestyle, productivity, etc.).

In [28]:
print('Andriod frequency table of the column Genre:')
print('\n')
display_table(android_final,9)
print('\n')

Andriod frequency table of the column Genre:


Tools : 8.449909747292418
Entertainment : 6.069494584837545
Education : 5.347472924187725
Business : 4.591606498194946
Productivity : 3.892148014440433
Lifestyle : 3.892148014440433
Finance : 3.7003610108303246
Medical : 3.531137184115524
Sports : 3.463447653429603
Personalization : 3.3167870036101084
Communication : 3.2378158844765346
Action : 3.1024368231046933
Health & Fitness : 3.0798736462093865
Photography : 2.944494584837545
News & Magazines : 2.7978339350180503
Social : 2.6624548736462095
Travel & Local : 2.3240072202166067
Shopping : 2.2450361010830324
Books & Reference : 2.1435018050541514
Simulation : 2.0419675090252705
Dating : 1.861462093862816
Arcade : 1.8501805054151623
Video Players & Editors : 1.7712093862815883
Casual : 1.7599277978339352
Maps & Navigation : 1.3989169675090252
Food & Drink : 1.2409747292418771
Puzzle : 1.128158844765343
Racing : 0.9927797833935018
Role Playing : 0.9363718411552346
Libraries & Demo : 0.936

The difference between the `Genres` and the `Category` columns is not crystal clear, but one thing we can notice is that the Genres column is much more granular (it has more categories). 

## Most Popular Apps by Genre on the App Store

One way to find out what genres are the most popular (have the most users) is to calculate the average number of installs for each app genre. 

In [29]:
genres_ios = freq_table(ios_final,11)

for genre in genres_ios: #Loop over each app genre in the freq_table
    total = 0 #Stores the sum of the user ratings
    len_genre = 0 #Stores the number of apps in each genre

    for app in ios_final:
        genre_app = app[11] #pull the genre of app in ios final

        if genre_app == genre: 
            num_ratings = int(app[5]) #pull the number of user ratings
            total += num_ratings #add up num of user ratings in the genre
            len_genre += 1 #counting the number of apps in the genre
        
    ios_average = total/len_genre #dividing the number of reviews/num of apps in each genre
    print (genre, ':', ios_average)

Social Networking : 71548.34905660378
Photo & Video : 28441.54375
Games : 22788.6696905016
Music : 57326.530303030304
Reference : 74942.11111111111
Health & Fitness : 23298.015384615384
Weather : 52279.892857142855
Utilities : 18684.456790123455
Travel : 28243.8
Shopping : 26919.690476190477
News : 21248.023255813954
Navigation : 86090.33333333333
Lifestyle : 16485.764705882353
Entertainment : 14029.830708661417
Food & Drink : 33333.92307692308
Sports : 23008.898550724636
Book : 39758.5
Finance : 31467.944444444445
Education : 7003.983050847458
Productivity : 21028.410714285714
Business : 7491.117647058823
Catalogs : 4004.0
Medical : 612.0


It looks like Navigation holds the most reviews on average. An app that is in the Navigation genre might be the reccomendation.

In [30]:
category_android = freq_table(android_final,1)

for category in category_android: #Loop over each app category in the freq_table
    total = 0 #Stores the sum of the user ratings
    len_category = 0 #Stores the number of apps in each category

    for app in android_final:
        app_category = app[1] #pull the category of app in android final

        if app_category == category: 
            installs = app[5] #pull the number of installs
            installs = installs.replace(',','')
            installs = installs.replace('+','')
            total += float(installs) #add up num of installs in the category
            len_category += 1 #counting the number of apps in the category
        
    android_average = total/len_category #dividing the number of reviews/num of apps in each category
    print (category, ':', android_average)

ART_AND_DESIGN : 1986335.0877192982
AUTO_AND_VEHICLES : 647317.8170731707
BEAUTY : 513151.88679245283
BOOKS_AND_REFERENCE : 8767811.894736841
BUSINESS : 1712290.1474201474
COMICS : 817657.2727272727
COMMUNICATION : 38456119.167247385
DATING : 854028.8303030303
EDUCATION : 1833495.145631068
ENTERTAINMENT : 11640705.88235294
EVENTS : 253542.22222222222
FINANCE : 1387692.475609756
FOOD_AND_DRINK : 1924897.7363636363
HEALTH_AND_FITNESS : 4188821.9853479853
HOUSE_AND_HOME : 1331540.5616438356
LIBRARIES_AND_DEMO : 638503.734939759
LIFESTYLE : 1437816.2687861272
GAME : 15588015.603248259
FAMILY : 3695641.8198090694
MEDICAL : 120550.61980830671
SOCIAL : 23253652.127118643
SHOPPING : 7036877.311557789
PHOTOGRAPHY : 17840110.40229885
SPORTS : 3638640.1428571427
TRAVEL_AND_LOCAL : 13984077.710144928
TOOLS : 10801391.298666667
PERSONALIZATION : 5201482.6122448975
PRODUCTIVITY : 16787331.344927534
PARENTING : 542603.6206896552
WEATHER : 5074486.197183099
VIDEO_PLAYERS : 24727872.452830188
NEWS_AND_

In [32]:
for app in android_final:
    if app[1] == 'COMMUNICATION' and (app[5] == '1,000,000,000+'
                                      or app[5] == '500,000,000+'
                                      or app[5] == '100,000,000+'):
        print(app[0], ':', app[5]) 

WhatsApp Messenger : 1,000,000,000+
imo beta free calls and text : 100,000,000+
Android Messages : 100,000,000+
Google Duo - High Quality Video Calls : 500,000,000+
Messenger – Text and Video Chat for Free : 1,000,000,000+
imo free video calls and chat : 500,000,000+
Skype - free IM & video calls : 1,000,000,000+
Who : 100,000,000+
GO SMS Pro - Messenger, Free Themes, Emoji : 100,000,000+
LINE: Free Calls & Messages : 500,000,000+
Google Chrome: Fast & Secure : 1,000,000,000+
Firefox Browser fast & private : 100,000,000+
UC Browser - Fast Download Private & Secure : 500,000,000+
Gmail : 1,000,000,000+
Hangouts : 1,000,000,000+
Messenger Lite: Free Calls & Messages : 100,000,000+
Kik : 100,000,000+
KakaoTalk: Free Calls & Text : 100,000,000+
Opera Mini - fast web browser : 100,000,000+
Opera Browser: Fast and Secure : 100,000,000+
Telegram : 100,000,000+
Truecaller: Caller ID, SMS spam blocking & Dialer : 100,000,000+
UC Browser Mini -Tiny Fast Private & Secure : 100,000,000+
Viber Mess

On average, communication apps have the most installs: 38,456,119. This number is heavily skewed up by a few apps that have over one billion installs (WhatsApp, Facebook Messenger, Skype, Google Chrome, Gmail, and Hangouts), and a few others with over 100 and 500 million installs.

If I removed all the communication apps that have over 100 million installs, the average would be reduced roughly ten times

In [33]:
under_100_m = []

for app in android_final:
    n_installs = app[5]
    n_installs = n_installs.replace(',', '')
    n_installs = n_installs.replace('+', '')
    if (app[1] == 'COMMUNICATION') and (float(n_installs) < 100000000):
        under_100_m.append(float(n_installs))
        
sum(under_100_m) / len(under_100_m)

3603485.3884615386

The same pattern applies to the video players category, which is the runner-up with 24,727,872 installs. The market is dominated by apps like Youtube, Google Play Movies & TV, or MX Player. The pattern is repeated for social apps (where there are giants like Facebook, Instagram, Google+, etc.), photography apps (Google Photos and other popular photo editors), or productivity apps (Microsoft Word, Dropbox, Google Calendar, Evernote, etc.).

Again, the main concern is that these app genres might seem more popular than they really are. Moreover, these niches seem to be dominated by a few giants who are hard to compete against.

The books and reference genre looks fairly popular as well, with an average number of installs of 8,767,811. It's interesting to explore this in more depth, since this genre has some potential to work well on the App Store, and my aim is to find an app genre that shows potential for being profitable on both the App Store and Google Play.

In [34]:
for app in android_final:
    if app[1] == 'BOOKS_AND_REFERENCE':
        print(app[0], ':', app[5])

E-Book Read - Read Book for free : 50,000+
Download free book with green book : 100,000+
Wikipedia : 10,000,000+
Cool Reader : 10,000,000+
Free Panda Radio Music : 100,000+
Book store : 1,000,000+
FBReader: Favorite Book Reader : 10,000,000+
English Grammar Complete Handbook : 500,000+
Free Books - Spirit Fanfiction and Stories : 1,000,000+
Google Play Books : 1,000,000,000+
AlReader -any text book reader : 5,000,000+
Offline English Dictionary : 100,000+
Offline: English to Tagalog Dictionary : 500,000+
FamilySearch Tree : 1,000,000+
Cloud of Books : 1,000,000+
Recipes of Prophetic Medicine for free : 500,000+
ReadEra – free ebook reader : 1,000,000+
Anonymous caller detection : 10,000+
Ebook Reader : 5,000,000+
Litnet - E-books : 100,000+
Read books online : 5,000,000+
English to Urdu Dictionary : 500,000+
eBoox: book reader fb2 epub zip : 1,000,000+
English Persian Dictionary : 500,000+
Flybook : 500,000+
All Maths Formulas : 1,000,000+
Ancestry : 5,000,000+
HTC Help : 10,000,000+
E

It seems there's still a small number of extremely popular apps that skew the average.

In [35]:
for app in android_final:
    if app[1] == 'BOOKS_AND_REFERENCE' and (app[5] == '1,000,000,000+'
                                            or app[5] == '500,000,000+'
                                            or app[5] == '100,000,000+'):
        print(app[0], ':', app[5])

Google Play Books : 1,000,000,000+
Bible : 100,000,000+
Amazon Kindle : 100,000,000+
Wattpad 📖 Free Books : 100,000,000+
Audiobooks from Audible : 100,000,000+


It looks like there are only a few very popular apps, so this market still shows potential. I'll try to get some app ideas based on the kind of apps that are somewhere in the middle in terms of popularity (between 1,000,000 and 100,000,000 downloads)

In [36]:
for app in android_final:
    if app[1] == 'BOOKS_AND_REFERENCE' and (app[5] == '1,000,000+'
                                            or app[5] == '5,000,000+'
                                            or app[5] == '10,000,000+'
                                            or app[5] == '50,000,000+'):
        print(app[0], ':', app[5])

Wikipedia : 10,000,000+
Cool Reader : 10,000,000+
Book store : 1,000,000+
FBReader: Favorite Book Reader : 10,000,000+
Free Books - Spirit Fanfiction and Stories : 1,000,000+
AlReader -any text book reader : 5,000,000+
FamilySearch Tree : 1,000,000+
Cloud of Books : 1,000,000+
ReadEra – free ebook reader : 1,000,000+
Ebook Reader : 5,000,000+
Read books online : 5,000,000+
eBoox: book reader fb2 epub zip : 1,000,000+
All Maths Formulas : 1,000,000+
Ancestry : 5,000,000+
HTC Help : 10,000,000+
Moon+ Reader : 10,000,000+
English-Myanmar Dictionary : 1,000,000+
Golden Dictionary (EN-AR) : 1,000,000+
All Language Translator Free : 1,000,000+
Aldiko Book Reader : 10,000,000+
Dictionary - WordWeb : 5,000,000+
50000 Free eBooks & Free AudioBooks : 5,000,000+
Al-Quran (Free) : 10,000,000+
Al Quran Indonesia : 10,000,000+
Al'Quran Bahasa Indonesia : 10,000,000+
Al Quran Al karim : 1,000,000+
Al Quran : EAlim - Translations & MP3 Offline : 5,000,000+
Koran Read &MP3 30 Juz Offline : 1,000,000+
H

# Conclusions
In this project, I analyzed data about the App Store and Google Play mobile apps with the goal of finding an app profile that can be profitable for both markets.

I concluded that taking a popular book (perhaps a more recent book) and turning it into an app could be profitable for both the Google Play and the App Store markets.